# This notebook will serve as the interactive notebook for leveraging the SNAP Graph Processing Tool using the subprocess library. 
- Allows for easier bulk processing of many interferograms, which is needed for non-zero closure phase correction (InSAR triplets)
- Allows for easier bulk processing of custom GRD-like products (will produce both calibrate and uncalibrated backscatter data for classification)

In [ ]:
# import dependencies
import os
from time import perf_counter
import subprocess

# Functions

In [ ]:
def project_dir(cwd, project_name):
    work_dir = os.path.join(cwd, project_name)
    os.makedirs(work_dir, exist_ok=True)
    data_dir = os.path.join(work_dir,'0_initial')
    os.makedirs(data_dir, exist_ok=True)

    return work_dir, data_dir

In [ ]:
# Helper functions
def bytes2snap(mem_bytes):
    """Convert memory size in bytes to SNAP-readable format (e.g., MB)."""
    return f"{int(mem_bytes / (1024 * 1024))}M"

def run_command(command):
    """Run a shell command and print the output."""
    try:
        subprocess.run(command, check=True)
        print(f"Command executed successfully: {' '.join(command)}")
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {e}")
        
def process_interferogram_individual(master, slave, output_dir):
    """Process two SLCs into an interferogram without a graph file."""
    coregistered_output = os.path.join(output_dir, f"{os.path.basename(master)}_{os.path.basename(slave)}_coregistered.dim")
    interferogram_output = os.path.join(output_dir, f"{os.path.basename(master)}_{os.path.basename(slave)}_interferogram.dim")

    # Step 1: Coregistration
    coregistration_cmd = [
        GPT_PATH,
        "Back-Geocoding",
        "-q", str(MAX_CORES),
        "-J-Xms2G",
        f"-J-Xmx{bytes2snap(MAX_MEM)}",
        f"-J-Dsnap.jai.defaultTileSize={TILE_SIZE}",
        f"-J-Dsnap.dataio.reader.tileWidth={TILE_SIZE}",
        f"-J-Dsnap.dataio.reader.tileHeight={TILE_SIZE}",
        "-J-Dsnap.jai.prefetchTiles=true",
        f"-Pmaster={master}",
        f"-Pslave={slave}",
        f"-PdemName=SRTM 1Sec HGT",
        f"-Poutput={coregistered_output}"  # Fixed '=' syntax
    ]
    print(f"Running Coregistration: {master} and {slave}")
    tic = perf_counter()
    run_command(coregistration_cmd)
    toc = perf_counter()
    print(f"Coregistration completed in {toc - tic:.2f}s: {coregistered_output}")

    # Step 2: Interferogram Formation
    interferogram_cmd = [
        GPT_PATH,
        "Interferogram",
        "-q", str(MAX_CORES),
        "-J-Xms2G",
        f"-J-Xmx{bytes2snap(MAX_MEM)}",
        f"-J-Dsnap.jai.defaultTileSize={TILE_SIZE}",
        f"-J-Dsnap.dataio.reader.tileWidth={TILE_SIZE}",
        f"-J-Dsnap.dataio.reader.tileHeight={TILE_SIZE}",
        "-J-Dsnap.jai.prefetchTiles=true",
        f"-Pinput={coregistered_output}",
        f"-Poutput={interferogram_output}"  # Fixed '=' syntax
    ]
    print(f"Running Interferogram Formation: {coregistered_output}")
    tic = perf_counter()
    run_command(interferogram_cmd)
    toc = perf_counter()
    print(f"Interferogram Formation completed in {toc - tic:.2f}s: {interferogram_output}")

    return interferogram_output


# Establish needed paths for notebook

In [ ]:
#establish working directory, where your SLCs were stored
# will also be where the processing steps for the interferograms and GRD-like products are stored
work_dir, data_dir = project_dir('/mnt/d/SabineRS', 'Sentinel-1')

In [ ]:
# get your path to the SNAP GPT install
# typically found in your 'esa-snap' install file in the 'bin' subdirectory
# Linux example: ~/esa-snap/bin/gpt
# WSL example: /mnt/c/Program Files/bin/gpt

# Define paths and parameters
GPT_PATH = '/mnt/c/Program Files/esa-snap/bin/gpt.exe'  # Path to SNAP's gpt executable
INPUT_DIR = data_dir  # Directory containing Sentinel-1 SLCs
OUTPUT_DIR = os.path.join(work_dir, 'test')  # Directory to save interferograms
os.makedirs(OUTPUT_DIR, exist_ok=True)
MAX_CORES = 12  # Adjust based on system capacity
MAX_MEM = 24 * 1024 * 1024 * 1024  # Maximum memory for SNAP in bytes
TILE_SIZE = 4096  # Tile size for SNAP

In [ ]:
# List all Sentinel-1 SLC files
slc_files = sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".zip")])
print("Available Sentinel-1 SLC Files:")
for i, file in enumerate(slc_files):
    print(f"{i + 1}: {file}")

# Select SLC pairs (e.g., manually choose indices for master and slave)
master_index = 0  # Update this to the index of the desired master file
slave_index = 1   # Update this to the index of the desired slave file

master_file = slc_files[master_index]
slave_file = slc_files[slave_index]

print(f"\nSelected Master: {master_file}")
print(f"Selected Slave: {slave_file}")

In [ ]:
# Example TOPSAR-Split cell, use this to build more cells for the other steps needed
# may make two notebooks; one for interferograms and the other for the custom backscatter products

# Define the command
cmd = [
    GPT_PATH,
    "TOPSAR-Split", # operation name
    "-Ssource=D:\\SabineRS\\Sentinel-1\\0_initial\\S1A_IW_SLC__1SDV_20170402T001815_20170402T001842_015958_01A507_82BB.zip", # SLC file
    "-PselectedPolarisations=VV,VH", # bands being processed
    #"-PwktAoi='POLYGON((NNN NNN, NNN NNN))'" # AOI
    "-t", "D:\\split_1.dim" # keep separate, output file location
]

# Run the command
try:
    result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully!")
except subprocess.CalledProcessError as e:
    print(f"Command failed with error code {e.returncode}")
    print(f"Error message: {e.stderr}")
